# Prepare lupus dataset for DM comparisons

Bulk data here: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE164457

In [3]:
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data_path = '/data_volume/memento/method_comparison/lupus/'


In [6]:
adata = sc.read(data_path + '../../lupus/Lupus_study_adjusted_counts.h5ad')

bulk = pd.read_csv(data_path + 'lupus_bulk.csv', index_col=0)

def get_sc_ind(x):
    
    if '-' in x:
        return x.split('-')[1]
    elif '_' in x:
        return x.split('_')[0]
    else:
        return x

meta = adata.obs[['ind_cov', 'Age', 'Sex', 'SLE_status']].drop_duplicates().reset_index(drop=True)
meta['ind'] = meta['ind_cov'].apply(get_sc_ind)

In [10]:
adata.obs.groupby(['ind_cov', 'cg_cov']).size().reset_index().head(30)

,ind_cov,cg_cov,0
0,1004_1004,B,49
1,1004_1004,NK,107
2,1004_1004,PB,0
3,1004_1004,Progen,3
4,1004_1004,Prolif,124
5,1004_1004,T4,1385
6,1004_1004,T8,578
7,1004_1004,cDC,32
8,1004_1004,cM,641
9,1004_1004,ncM,64


In [14]:
min(5, 3)

3

In [11]:
sc.pp.subsample(adata, n_obs=1000)

In [13]:
sc.pp.subsample(adata, n_obs=10000)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [12]:
a.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

data_path = '/data_volume/memento/method_comparison/lupus/'

### Read single cell and bulk data

adata = sc.read(data_path + '../../lupus/Lupus_study_adjusted_counts.h5ad')

bulk = pd.read_csv(data_path + 'lupus_bulk.csv', index_col=0)

def get_sc_ind(x):
    
    if '-' in x:
        return x.split('-')[1]
    elif '_' in x:
        return x.split('_')[0]
    else:
        return x

meta = adata.obs[['ind_cov', 'Age', 'Sex', 'SLE_status']].drop_duplicates().reset_index(drop=True)
meta['ind'] = meta['ind_cov'].apply(get_sc_ind)

sc_inds = set(meta['ind'].tolist())

bulk_inds = set(bulk.columns.str.split('_').str[1].tolist())

inds = list(sc_inds & bulk_inds)

meta = meta[meta['ind'].isin(inds)]

genes = list(set(bulk.index) & set(adata.var.index))

## Create CD14 vs CD4 comparisons

### Sample individuals

# For now, we'll stick to comparing CD14 vs CD4 cells

sampled_inds = np.random.choice(inds, 4)

### Create single cell data and pseudobulks

adata.obs['ind'] = adata.obs['ind_cov'].apply(get_sc_ind)

sampled_adata = adata[adata.obs['ind'].isin(sampled_inds) & adata.obs['cg_cov'].isin(['T4', 'cM']), genes]

pseudobulks = []
names = []
adata_list = []
for ind in sampled_inds:
    for ct in ['T4', 'cM']:
        ind_ct_adata = sampled_adata[(sampled_adata.obs['ind']==ind) & (sampled_adata.obs['cg_cov']==ct)].copy()
        sc.pp.subsample(ind_ct_adata, n_obs=100)
        adata_list.append(ind_ct_adata.copy())
        pseudobulks.append( ind_ct_adata.X.sum(axis=0).A1)
        names.append(('CD14' if ct == 'cM' else 'CD4') + '_' + ind )
sc_data = sc.AnnData.concatenate(*adata_list)
pseudobulks = np.vstack(pseudobulks)
pseudobulks = pd.DataFrame(pseudobulks.T, columns=names, index=genes)

pseudobulks.to_csv(data_path + 'T4_vs_cM.pseudobulk.csv')

sc_data.write(data_path + 'T4_vs_cM.single_cell.h5ad')

### Select bulk data

names = []
for ind in sampled_inds:
    for ct in ['T4', 'cM']:
        
        name = ('CD14' if ct == 'cM' else 'CD4') + '_' + ind
        names.append(name)
bulk.loc[genes, names].to_csv(data_path + 'T4_vs_cM.bulk.csv')

SyntaxError: invalid syntax (1523107730.py, line 40)